# Analysing data with Python and MNE

_Sebo Uithol, PhD, June 2023. Thanks to Jeshua Tromp and David Vogelsang for their help setting up this Jupyter Notebook._

Welcome to the practical session EEG data analysis using MNE! This practical session is part of the course Experimentation II at Leiden University. In this session we will study the EEG signals during an Eriksen Flankers Task. We will get acquainted with the basic preprocessing steps, the analysis of the data and the interpretation of the results.

The data that we will be analysing was recorded at Leiden University, with the same equipment that we have used during the data acquisition practical.

The data was acquired during an Eriksen flankers task. This is a conflict task that belongs to the same family of paradigms as the Stroop and Simon task. Participants are instructed to respond as fast as possible to the direction of the central stimulus in an array like this by pressing a response button with the corresponding index finger (in this case the right index finger) The experiment was a bit more complex than the standard Flankers task, but we will ignore this here.

Flanking stimuli around the target are irrelevant, but can prime the correct response (congruent), or the opposite response (incongruent). Immediately after stimulus presentation, the attentional spotlight (the area of stimulus information that is selected for further processing) has not yet been narrowed, so the dominant information derives from the flankers, which may point at the wrong response. As the spotlight is narrowed, the support for the correct response increases. In the ERP you are likely to see initial signs of motor preparation for the hand indicated by the flankers (LRP) and signs of error detection processes (ERN). In addition, the ERP will show large components such as the P300. Prior to the P300, there is a weak negative component that is sensitive to the presence of conflict in the decision processes (N200). We will see to what extent these waves can be identified and quantified.

### About MNE
MNE (MEG & EEG) is a python toolbox. Both python and MNE are free, which makes it an analysis combination that is rapidly gaining ground. Installing and setting it up can be a bit more tricky, though, as there is a web of versions, libraries, dependencies, etc, that can change quickly.

### About this tutorial
This tutorial takes you through the absolute basic steps of EEG data analysis using MNE. MNE has an excellent website (https://mne.tools/stable/auto_tutorials/index.html) with all the commands explained, and also sample scripts showing how these commands can be put to use. I encourage you to look it up, and incorporate the commands you find there in the script below. At the bottom of this tutorial there is room to add your own commants.

### Importing
Before we can start loading the data, we need to have the proper tools in place. Obviously we need to import MNE. Next we import matplotlib, for data visualisation, pandas for data analysis, and numpy for working with data structures. If you don't like typing the same words over and over again, you can import libraries as a shorter name, for instance 'pandas' as 'pd'. This saves time later on, but it is optional.

In [1]:
import matplotlib
import pandas as pd
import numpy as np
%pip install mne
import mne
%pip install python-picard

Note: you may need to restart the kernel to use updated packages.


### Loading the data
Now we want to create an array with the content of the eeg data file. We do this using the command read_raw_bdf. This command is part of the MNE toolbox, and within MNE of the io (input-output) library, so the full command will read as: nameArray = mne.io.read_raw("file name"). We set preload to _True_, which means that the entire file (390MB will be loaded into the memory. This will result in faster processing, but requires more RAM. If your computer does not have much memory, you may want to set this to _False_.

In [2]:
filename = "Flanker_EEG_52_12052022.bdf"

raw_eeg = mne.io.read_raw_bdf(filename, preload=True)

Extracting EDF parameters from /Users/sebouithol/DataspellProjects/Experimentation2DataAnalysis/Experimentation2/Flanker_EEG_52_12052022.bdf...
BDF file detected


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sebouithol/DataspellProjects/Experimentation2DataAnalysis/Experimentation2/Flanker_EEG_52_12052022.bdf'

We have managed to import the eeg data in python. At this point we may want to check what kind of data we have imported, so we use the command _info_. Python is an object-oriented programming language, so we can use our variable name, raw_eeg, and add .info to it.

In [ ]:
raw_eeg.info

The info displayed above is what is stored in the header of the file. You can see, for instance, that 72 electrodes have been used (twice the number that we've used in the data acquisition practical); That the recording took almost an hour, and that the analogue signal was sampled with a 512Hz frequency.
The data file itself is little more than the measured potential of a number of electrodes sampled at given frequency, stored as a very large 2-dimensional matrix. The size if the matrix is the number of channels for one dimension, and the sampling frequency timed the duration of the recording in seconds.

To make sense of this matrix, we will also have to specify what the locations of each of the electrodes is. Luckily we do not have to do this by hand. MNE has dictionaries (or rather atlases) of standard systems. Ours is called _biosemi64_: BioSemi is the brand of the EEG setup; 64 denotes the number of channels. If we use this atlas, the location of each of the electrodes is provided. Additionally, we will specify that EXG1 through EXG6 are not used for recording the eeg signal, but were additional electrodes used for reference and tracking eye movements (eog channels). We will drop EXG7 and EXG8, as they were not used during the recording.

In [ ]:
biosemi_montage = mne.channels.make_standard_montage("biosemi64")
raw_eeg.set_montage(biosemi_montage, on_missing='ignore')
raw_eeg.set_channel_types(
    {'EXG1': 'eog', 'EXG2': 'eog', 'EXG3': 'eog', 'EXG4': 'eog', 'EXG5': 'eog', 'EXG6': 'eog'})
raw_eeg.drop_channels(['EXG7', 'EXG8'])  # Unused channels

If you want to check the location of the sensors, you can use the plot_sensors command. Do you recognise the naming and location convention?

In [ ]:
raw_eeg.plot_sensors(show_names=True)
#fig = raw.plot_sensors("3d")

We have loaded the data, and specified some of the characteristics. Now it is time to preprocess the data.

# Preprocessing
EEG preprocessing encompasses multiple steps. Generally, these steps are different in every experiment, as they are dependent on the research question, the recording setup, and the subsequent analysis steps. So there is no one-size-fits-all method. The steps below are common in many analysis pipelines.

### Downsampling
In the output above we saw all that the signal was sampled with a 512Hz frequency. For our purpose this is a bit too high, which will make files unnecessarily big, and computations slow. So we will _downsample_ the data. This will be done using the resample command. We will have to specify the sfreq for this. Note that we could have recorded this data with a sampling frequency of 250 from the start, which would have saved us the downsampling step. Yet, this latter option would have resulted in data of lesser quality. Do you understand why? Have a look at the command below. Do you understand the syntaxis? What is raw_eeg doing at both sides of the '=' sign?

In [ ]:
raw_eeg = raw_eeg.resample(sfreq=250)

If you want to check whether it was successful you can use the .info command again

In [ ]:
raw_eeg.info

### Rereferencing
You may remember from the lecture that the potential that is recorded at each electrode is not an absolute value, but always a difference between that electrode and another electrode. You may also remember that during the acquisition we have used two electrodes named 'drl' and 'cms' as a reference during recording, and that we placed two electrodes on the mastoids in addition. Since drl and cms were close to the active electrodes, they are not an ideal reference for analysing the data. The mastoids are further away from the active electrodes, and not near the brain, which will make them a superior reference. We will set the average of these two channels as explicit reference for all the other electrodes. Can you explain the procedure? Could we have placed cms and drl directly on the mastoids?


In [ ]:
raw_eeg.set_eeg_reference(ref_channels=['EXG5', 'EXG6'])  # Take average of mastoids as reference


Now let's have a quick inspection of what the raw data looks like.

In [ ]:
mne.viz.plot_raw(raw_eeg)

Ok, it is clearly data, but it does not very  look pretty. It still contains various types of noise, and at multiple places the recorded value is too small or large to be displayed in the given range. In different steps we will correct these problems.

### Filtering
The EEG signal can be constructed as an assembly of sinus waves of different frequencies, phase, and strength (a Fourrier Analysis). Of some of these frequencies, the very low and very high ones, we can be sure that they do not correlate to brain activity. Low frequencies may be attributed to amplifier drift, or sweat. We use a so-called _high-pass_ filter to discard this frequencies High pass means that frequencies higher than a certain threshold can 'pass' the filters; lower frequencies are filtered out.
Higher frequencies we filter out using a _low-pass filter_. There is some controversy around the source of high frequencies in the EEG signal. We are certain that line noise (50Hz in Europe) and muscle activation contribute to this frequency range, but there seems to be evidence that higher frequencies can also originate from the brain. So you have to choose the threshold frequency of your low-pass filter wisely, dependent on your research question.

Note that the names of the two filters always cause confusion. The value of a low-pass filter is always higher than that of a high-pass filter. Check whether you can explain why.

Here we will use a high pass setting of 0.1 Hz. This is relatively conservative filtering (everything slower than 10 second oscillations will be removed), but choose this setting because our independent component analysis will work better this way. We can of course choose to do another filtering step after the ICA. If you try you will see that your data will already look a bit better.

In [ ]:
low_cut = 0.1
high_cut = 30
filtered_eeg = raw_eeg.filter(low_cut, high_cut, fir_design='firwin')


Let's have a look at what the data looks like now. Use the same plot command as above, but this time plot the new array.

In [ ]:
mne.viz.plot_raw(filtered_eeg)

This already looks a bit better, but we still need to remove the artifacts that originate from eye movements (blinks and saccades). We will do this using an _Independent Component Analysis_.




#### Independent Component Anlysis
To understand an Independent Component Analysis (ICA), think of a number of musicians playing in a room, while being recorded by multiple microphones. Each microphone picks up the sound of all the musical instruments, but, importantly, at different strengths. An ICA is a mathematical procedure to re-construct the sound of each individual instrument based on the signals of each of the microphones (test yourself: what problem does an ICA have to solve?). The number of instruments that can thus be isolated is limited by the number of microphones.
Back to EEG data preprocessing: The raw signal is composed of proper brain activation as well as artifacts such as eye-blinks and saccades. What we want to do is identify components that can be related to occular artifacts, and then subtract these components from the raw signal. What would be left is a signal that does not include the contribution of these noise components. Here the number of components that can be estimated is limited by the number of electrodes. Since this data was recorded using 64 channels, we have ample components at our disposal.


In [ ]:
from mne.preprocessing import ICA
method = 'picard'

n_components = 20 # select n_components by explained variance of PCA
random_state = 23 # Random seed

ica = ICA(n_components=n_components, method=method, random_state=random_state)
ica.fit(filtered_eeg, reject=None)

Now let us plot these components for a visual inspection. The components are ranked ordinally, the first component explaining the most of the variance in the data, the last component the least. We can plot the contribution of the different components over time, but also the location of the component.

In [ ]:
ica.plot_components()
ica.plot_sources(filtered_eeg)


For best results we look at both the components and the sources. Note that the sources graph does not display the electrodes, but the different components. The graph indicates how much of the (in our case filtered eeg) signal is explained by each component. What component correlates with eye blinks, you think? Why? What component could correlate to a bad sensor connection? Why? We will remove it from the data.

Fill in the components number in the brackets below. Mind that the first member in an array in python is referred to as 0 (unlike for instance MatLab and R, where the first member is 1). Type in the components between the square brackets, separated by a comma

In [ ]:
ica.exclude = []
cleaned_eeg = ica.apply(filtered_eeg)


In [ ]:
mne.viz.plot_raw(cleaned_eeg)



### Filtering again
That looks a whole lot better! Remember that we filtered quite conservatively because of the ICA that we planned to do. We can do another, slightly less conservative round of filtering to get rid of the frequencies between 0.1 and 1 Hz. They are certainly not related to brain activity, so better discarded. These steps and settings can  be changed and even repeated, as we do here, as long as you are transparent about it when publishing the results.

So now we run another band pass filter, but this time with the low_cut set to 1, and we plot the results immediately.


In [ ]:
low_cut = 1
high_cut = 30
cleaned_eeg = cleaned_eeg.filter(low_cut, high_cut, fir_design='firwin')
mne.viz.plot_raw(cleaned_eeg)

This is starting to look like nice usable data! Now it is time to relate it to behavioural events

### Loading the behavioural data

Because MNE is a python toolbox, the possibilities to combine eeg with behavioural data are unlimited. In this case, the behavioural data is stored in a separate text file. I will not explain this step in detail, as it is very specific to the experiment, but have a look at the code, and see whether you understand what is happening. Crucial is that in this task on half of the trials the participant was given an alert on beforehand, and on half of the trials this alert was absent, like with the classical Flankers task.

In [ ]:
fileNameBehav = "flankertask_eyetracker_52_202205121431_format.txt"

# Load in behavioral data and make numpy array of trialtype
trials = pd.read_csv(fileNameBehav, sep=",")
trials['RT'].replace('None', np.nan, inplace=True)  # Replace "none" answers with "nan" for int processing
trials['RT'] = pd.to_numeric(trials['RT'], errors='coerce', downcast='integer')

trials['condition'] = trials['trialtype'] + ' ' + trials[
    'alert']  # Make condition column, with alert and trialtype
trials_num = trials[['condition']].to_numpy()

# Change coding of trialtype, and reshape to appropriate dimension for later processing
trials_num = np.where(trials_num == "congruent alert", 1, trials_num)
trials_num = np.where(trials_num == "congruent no_alert", 2, trials_num)
trials_num = np.where(trials_num == "incongruent alert", 3, trials_num)
trials_num = np.where(trials_num == "incongruent no_alert", 4, trials_num)
trials_num = np.where(trials_num == "catch alert", 5, trials_num)
trials_num = trials_num.reshape(720)

# Transfer the stimulus EEG triggers to an MNE event file
events = mne.find_events(raw_eeg, uint_cast=True, consecutive=False)

# Delete all markers except for "alert_onset" (34) --> we will use that to orient ourselves
# Note that there were also "alert_onset" triggers in no_alert trials, these were just silent
events_trials = mne.pick_events(events, exclude=[4, 8, 16, 33, 35, 36, 253])

# Merge trials_num and events_trials: replace third column np array with trials_num that contain trial condition
events_trials[:, 2] = trials_num

# alert-centered to flanker centered (+500)
events_trials[:, 0] = events_trials[:, 0] + (500 * (raw_eeg.info['sfreq'] / 1000))
# Dictionary of numbers
event_dict = {"congruent alert": 1, "congruent no_alert": 2,
              "incongruent alert": 3, "incongruent no_alert": 4,
              "catch alert": 5}

The behavioural data tells us what event occured at what time. We can use it to cut the continuous signal into chunks of data that are linked to one of the experimental condition. We call these chunks _epochs_. We use the behavioural data that we have loaded above to define the epochs. We specify how much time before, and how much time after the event we want to include into our epoch.


In [ ]:
tmin=-0.6
tmax = 0.7  # epoch from 0.6s before flanker onset to 0.7s after it

epochs = mne.Epochs(cleaned_eeg, events_trials, event_dict, tmin, tmax, baseline=None,
                    metadata=trials, preload=True)

We can now plot the epochs using the epochs.plot command (mind the capital E here!)

In [ ]:
epochs.plot(events=events)
#mne.Epochs.plot(epochs)

Now it is time to make averages of all epochs belonging to one condition.

In [ ]:
congNoAl = epochs["congruent no_alert"].average()
incongNoAl = epochs["incongruent no_alert"].average()
congAl = epochs["congruent alert"].average()
incongAl = epochs["incongruent alert"].average()


and again, we can plot them:

In [ ]:
congNoAl.plot()



Look at the plots carefully. What is shown is the ERP for every electrode overlayed on top of each other. Two observations may need explication. 1) The ERP is visible in every electrode, especially the positive peak at 400ms. This is inherent to EEG data and its poor spatial resolution. If we want to do source localisation we will have to use the difference between the ERP peaks and troughs of the various electrodes. You can see that during the positive peak at around 200ms, the spread over the scalp is quite good, with the greenish tints (frontal electrodes) are more positive thah the blue/purpel electrodes (what does that mean?). 2) The ERP components are much larger than the noise before 0s. This means that our recording and preprocessing step was succesfull and that the signal-to-noise ration is likely to be good enough to do analyses.

In [ ]:
evokeds = dict(congruentAlert=congAl, incongruentAlert=incongAl, congruentNoAlert=congNoAl, incongruentNoAlert=incongNoAl)
picks = ["Cz"]
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine="mean")

We can see that the difference between alert and no alert conditions is big at around 300ms before stimulus onset, but virtually absent in the peaks +400ms. In this peak we can see a difference between the congruent and incongruent conditions, irrespective of the alert.

The Tutorial ends here. If you want you can play with the settings of the commands above, or add new commands below. I started a time-frequency analysis below, but it is not yet the way I want it to be. You can play with it and see what it does. Alternatively, you can add your own commands that you looked up at the MNE website.

In [ ]:
raw_eeg.compute_psd(fmax=50).plot

In [ ]:
raw_eeg.compute_psd().plot()

In [ ]:
frequencies = np.arange(5, 30, 2)
power, itc = mne.time_frequency.tfr_morlet(epochs, n_cycles=2, return_itc=True,
                                           freqs=frequencies, decim=3, average=True)

In [ ]:
power.plot()